# Import Library

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVR
import numpy as np
import pickle
import joblib

# Load dataset

In [ ]:
# Load data dari Excel
data = pd.read_excel('/content/dataset1.xlsx')

data.head(9)

,x1,x2,x3,x4,x5,x6,y
0,SBY - LBR,9.279396,12.110000,0.530000,7.000000,1.070000,7390
1,LBR - SBY,9.623077,8.600000,2.748000,7.440000,1.180000,7770
2,SBY - LBR,9.912903,8.125000,0.956250,7.287500,0.850000,7860
3,LBR - SBY,9.388462,5.500000,1.116667,6.940000,0.833333,6713
4,SBY - LBR,10.212500,9.571429,0.638571,5.522857,0.657143,7749
5,LBR - SBY,9.260000,6.285714,0.291429,8.050000,0.685714,7110
6,SBY - LBR,9.448718,8.888889,0.362222,5.411111,0.622222,7620
7,LBR - SBY,9.687879,4.714286,0.278571,7.271429,0.700000,6800
8,SBY - LBR,9.103571,7.125000,0.308750,7.425000,0.600000,7800


#Normalisasi dan split dataset

In [ ]:
# Pisahkan fitur dan target
X = data[['x1', 'x2', 'x3', 'x4', 'x5', 'x6']]
y = data['y']

In [ ]:
# Pengkodean one-hot untuk fitur 'route'
X_encoded = pd.get_dummies(X, columns=['x1'], drop_first=True)

# Tampilkan hasil one-hot encoding
print("Hasil One-Hot Encoding:")
print(X_encoded.head())  # Tampilkan lima baris pertama dari data yang telah di-encode

Hasil One-Hot Encoding:
          x2         x3        x4        x5        x6  x1_SBY - LBR
0   9.279396  12.110000  0.530000  7.000000  1.070000             1
1   9.623077   8.600000  2.748000  7.440000  1.180000             0
2   9.912903   8.125000  0.956250  7.287500  0.850000             1
3   9.388462   5.500000  1.116667  6.940000  0.833333             0
4  10.212500   9.571429  0.638571  5.522857  0.657143             1


In [ ]:
# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

# Scaling data
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
joblib.dump(scaler, 'scaler.pkl')

['scaler.pkl']

In [ ]:
# menampilkan hasil normalisasi pada data training
print(X_train_scaled[:5])

[[0.19703459 0.36800958 0.0908498  0.24254445 0.0703125  1.        ]
 [0.46704848 0.36800958 0.29025345 0.32986045 0.07714844 1.        ]
 [0.3452488  0.59163262 0.11077981 0.         0.08246528 1.        ]
 [0.18083768 0.13663896 0.13119642 0.07814949 0.08671875 0.        ]
 [0.67323516 0.67816393 0.21377988 0.01345823 0.1015625  1.        ]]


In [ ]:
# menampilkan hasil normalisasi pada data testing
print(X_test_scaled[:5])

[[ 0.35842946  0.13482784  0.21111762  0.17243357 -0.015625    0.        ]
 [ 0.60747961 -0.04628418  0.13817156  0.43911298  0.265625    1.        ]
 [ 0.6941869   0.11671664  0.04871945  0.09501051  0.1328125   1.        ]
 [ 0.58068127  0.25995979  0.18415613  0.0150849   0.08025568  1.        ]
 [ 0.82024362  0.03014509  0.07516462  0.3158096   0.25260417  1.        ]]


# Modeling

In [ ]:
# Model Training dan Evaluasi dengan Keempat Kernel dengan GridSearchCV
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
models = {}

for kernel in kernels:
    svr_model = SVR(kernel=kernel)

    # Set parameter grid untuk kernel tertentu
    param_grid = {}
    if kernel == 'linear':
        param_grid = {'C': [0.1, 1, 10]}
    elif kernel == 'poly':
        param_grid = {'C': [0.1, 1, 10], 'degree': [2, 3, 4], 'coef0': [0.0, 0.1, 0.5], 'gamma': [0.01, 0.1, 1]}
    elif kernel == 'rbf':
        param_grid = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1]}
    elif kernel == 'sigmoid':
        param_grid = {'C': [0.1, 1, 10], 'gamma': [0.01, 0.1, 1], 'coef0': [0.0, 0.1, 0.5]}

    # GridSearchCV untuk mencari parameter optimal pada kernel tertentu
    grid_search = GridSearchCV(svr_model, param_grid, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train_scaled, y_train)

    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test_scaled)

    mape = np.mean(np.abs((y_test - y_pred) / y_test)) * 100

    models[kernel] = best_model
    print(f"Kernel: {kernel}")
    print(f"Best Parameters: {grid_search.best_params_}")
    print(f"Mean Absolute Percentage Error: {mape}")
    print("-----------------------")

    # Simpan model ke dalam file menggunakan joblib
    filename = f'model_{kernel}.pkl'
    joblib.dump(best_model, filename)

# Model terbaik berdasarkan R-squared tertinggi
# Model terbaik berdasarkan MAPE terendah
best_model_mape = models[min(models, key=lambda k: np.mean(np.abs((y_test - models[k].predict(X_test_scaled)) / y_test)) * 100)]

print("Model Terbaik dengan MAPE terendah:", best_model_mape.kernel)
mape_best_model = np.mean(np.abs((y_test - best_model_mape.predict(X_test_scaled)) / y_test)) * 100
print("MAPE terbaik:", mape_best_model)

Kernel: linear
Best Parameters: {'C': 10}
Mean Absolute Percentage Error: 3.0448920547548033
-----------------------
Kernel: poly
Best Parameters: {'C': 1, 'coef0': 0.5, 'degree': 3, 'gamma': 1}
Mean Absolute Percentage Error: 3.043486244940877
-----------------------
Kernel: rbf
Best Parameters: {'C': 10, 'gamma': 1}
Mean Absolute Percentage Error: 3.048020807648127
-----------------------
Kernel: sigmoid
Best Parameters: {'C': 10, 'coef0': 0.0, 'gamma': 1}
Mean Absolute Percentage Error: 3.0296954726319285
-----------------------
Model Terbaik dengan MAPE terendah: sigmoid
MAPE terbaik: 3.0296954726319285


# Rumus Setiap Model

**Linear**

In [ ]:
# Load model dari file yang telah diexport
loaded_model = joblib.load('model_linear.pkl')

# Menampilkan bobot (weights) dan bias (intercept) dari model SVR linear yang dimuat kembali
w = loaded_model.coef_
b = loaded_model.intercept_
print("Bobot: ", w)
print("Bias: ", b)
print("-----------")
print("Rumus Model SVR dengan Kernel Linear:")
print("f(x) = ", end="")
for i in range(len(w)):
    if i > 0:
        print(" + ", end="")
    print(f"({w[i]} * x{i+1})", end="")
print(f" + ({b})")

Bobot:  [[-10.87700126   7.53222591  -3.03935824  -0.21988458   7.98098035
   11.98470078]]
Bias:  [7447.7666779]
-----------
Rumus Model SVR dengan Kernel Linear:
f(x) = ([-10.87700126   7.53222591  -3.03935824  -0.21988458   7.98098035
  11.98470078] * x1) + ([7447.7666779])


**Polinomial**

In [ ]:
loaded_model_poly = joblib.load('model_poly.pkl')

# Mendapatkan nilai gamma, coef0, dan derajat polinomial dari model yang dimuat kembali
gamma = loaded_model_poly.gamma
coef0 = loaded_model_poly.coef0
degree = loaded_model_poly.degree

print("gamma: ", gamma)
print("coef0: ", coef0)
print("degree: ", degree)
print("-----------")

print("Rumus Model SVR dengan Kernel Polinomial:")
print(f"f(x) = (gamma * <x, x'> + {coef0})^{degree}")

gamma:  1
coef0:  0.5
degree:  3
-----------
Rumus Model SVR dengan Kernel Polinomial:
f(x) = (gamma * <x, x'> + 0.5)^3


**RBF**

In [ ]:
loaded_model_rbf = joblib.load('model_rbf.pkl')

# Mendapatkan nilai gamma dari model yang dimuat kembali
gamma = loaded_model_rbf.gamma
print("gamma: ", gamma)
print("-----------")

print("Rumus Model SVR dengan Kernel RBF:")
print(f"f(x) = sum_i (alpha_i * exp(-{gamma} * ||x - x_i||^2)) + b")

gamma:  1
-----------
Rumus Model SVR dengan Kernel RBF:
f(x) = sum_i (alpha_i * exp(-1 * ||x - x_i||^2)) + b


**Sigmoid**

In [ ]:
# Load model dari file yang telah diexport (misalnya, model dengan kernel sigmoid)
loaded_model_sigmoid = joblib.load('model_sigmoid.pkl')

# Mendapatkan nilai gamma dan coef0 dari model yang dimuat kembali
gamma = loaded_model_sigmoid.gamma
coef0 = loaded_model_sigmoid.coef0

print("gamma: ", gamma)
print("coef0: ", coef0)
print("-----------")


print("Rumus Model SVR dengan Kernel Sigmoid:")
print(f"f(x) = tanh({gamma} * <x, x'> + {coef0})")

gamma:  1
coef0:  0.0
-----------
Rumus Model SVR dengan Kernel Sigmoid:
f(x) = tanh(1 * <x, x'> + 0.0)


# Inferensi ke data baru


In [ ]:
# 1 = SBY-LBR, 2= LBR SBY
data_baru_scaled = np.array([[1, 9.3, 5.5, 1.11, 6.94, 0.83]]).reshape(1, -1)
scaler = MinMaxScaler()
data_baru_scaled =  scaler.fit_transform(data_baru_scaled)

In [ ]:
# Load model terbaik yang telah Anda simpan menggunakan pickle
best_model_filename = 'model_sigmoid.pkl'
best_model = joblib.load(best_model_filename)

In [ ]:
# Lakukan prediksi nilai target (ship fuel consumption) pada data baru
prediction = best_model.predict(data_baru_scaled)

# Cetak hasil prediksi
print("Hasil Prediksi Ship Fuel Consumption:")
print(prediction[0])

Hasil Prediksi Ship Fuel Consumption:
7448.612068146467
